In [1]:
from data import data_loader
from all_coefs import all_coefs,load_coefs
from coefs import weight_matrix
from spread import spread_manager
from revert import df_to_inv_mom,df_to_dict
from pnl import pnl

from scipy.optimize import minimize

import pandas as pd
import datetime as dt
import numpy as np
import tensorflow as tf

import time

In [2]:
data=data_loader('2024-01-01') #data_obj with all precomputes
price_data=data.load_price()

price_dict=df_to_dict(price_data) #Price Dictionary

In [3]:
"""
#CREATE COEF MATRIX
coefs=all_coefs(data)
coefs.run(t_stat=0.25,weight_imb=0.5,corr_thresh=0.85,weight_type='std')
print(coefs.errors())
coefs.coefs()
coefs.dump_matrix('t025imb05corr085typestd.pkl')
"""

"\n#CREATE COEF MATRIX\ncoefs=all_coefs(data)\ncoefs.run(t_stat=0.25,weight_imb=0.5,corr_thresh=0.85,weight_type='std')\nprint(coefs.errors())\ncoefs.coefs()\ncoefs.dump_matrix('t025imb05corr085typestd.pkl')\n"

In [4]:
coef_matrix=load_coefs(data,'t025imb05corr085typeabs.pkl') #Load COEF MATRIX

dates_used=data.all_dates()[data.all_dates()>=pd.Timestamp(dt.datetime(2021,12,31))] #ISOLATE DATES FOR BACKTEST

In [5]:
spreads=spread_manager(price_data,coef_matrix)
spreads.calc_costs()
signals=spreads.raw_signal()

z_score=spreads.new_z_score()
rev=df_to_inv_mom(z_score)

filterer=spreads.signal_z_score_filter()

In [6]:
signal_dict=df_to_dict(signals)
rev_dict=df_to_dict(rev)
filter_dict=df_to_dict(filterer)

In [7]:
#ALL VALUES THAT NEED DATES

date_old=dates_used[0]
date_new=dates_used[1]

weight_data=weight_matrix(price_dict[date_old],coef_matrix)

betas=data.load_data(date_old,dtype='beta')
covs=data.load_data(date_old,dtype='cov')

In [8]:
initial_weights=price_dict[dates_used[0]].copy()
initial_weights.loc[:]=0
initial_weights

Ticker
FOX      0.0
NOW      0.0
AMCR     0.0
JPM      0.0
KO       0.0
        ... 
CCI      0.0
APTV     0.0
LVS      0.0
MOH      0.0
CONST    0.0
Name: 2021-12-31 00:00:00, Length: 498, dtype: float64

In [109]:
initial_value=100e6

#Signal
c_sig=signal_dict[date_old][filter_dict[date_old]]

#Reversion Speed
c_rev=rev_dict[date_old][filter_dict[date_old]]
lam_1=0.1

#Weight Matrix
c_A=weight_data[filter_dict[date_old]].T

#Initial Weights DIRECT INPUT
c_w0=initial_weights.copy()
c_w0.drop('CONST',inplace=True)
lam_2=0.2

#COV Matrix
c_cov=data.load_data(date_old,dtype='cov')
lam_3=0.3

#Betas
c_beta=data.load_data(date_old,dtype='beta')

theta_1=0.2 #Cash Neutrality
theta_2=0.2 #Beta Neutrality
theta_3=0.5 #Max Total Capital to All Trades
theta_4=0.01 #Max Cap per Trade


In [10]:
#INITIALIZE TRADES (RANDOM INPUT)
t_init=np.random.normal(size=sum(filter_dict[date_old]))
t_init=t_init/(2.5*np.sum(np.abs(t_init)))

In [11]:
t=t_init #TRANSLATION FOR OBJECTIVE FUNCTION

In [61]:
#Term 1 (MAX RETURN)
print(t_init@c_sig)
#Term 2 (MAX REV)
print(lam_1*(t_init@c_rev))
#Term 3 (TRANSACTION COSTS)
print(lam_2*sum(abs((c_A@t_init)-c_w0)))
#Term 4 (Volatility)
print(lam_3*((c_A@t_init).T@c_cov@(c_A@t_init)))

-0.00036502333747282976
5.2769105665952803e-05
0.037701968154873144
9.303852121787058e-07


In [72]:
((c_A@t_init).T@c_cov@(c_A@t_init))

3.101284040595686e-06

In [132]:
t = tf.Variable(t_init, dtype=np.float64)
def obj_func(t):
    ret = tf.reduce_sum(t * c_sig)
    rev = lam_1 * tf.reduce_sum(t * c_rev)
    c_w=tf.matmul(c_A,tf.expand_dims(t, axis=1))
    tcosts = lam_2 *tf.reduce_sum(tf.abs(c_w-tf.expand_dims(c_w0,axis=1)))
    vol=lam_3*tf.reduce_sum(tf.matmul(tf.matmul(c_w,c_cov,transpose_a=True),c_w))
    return -(ret + rev - tcosts - vol)

In [128]:
def compute_gradients(t):
    with tf.GradientTape() as tape:
        loss = obj_func(t)
    return tape.gradient(loss, t)

In [134]:
optimizer = tf.optimizers.Adam(0.0001)

n_epochs = 1000
for epoch in range(n_epochs):
    grads = compute_gradients(t)
    optimizer.apply_gradients(zip([grads], [t]))
    t.assign(tf.clip_by_value(t, -theta_4, theta_4))
    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Objective value: {obj_func(t).numpy()}")

# Final optimized parameters
print("Optimized parameters:", t.numpy())

Epoch 0, Objective value: -0.9304960939395481
Epoch 10, Objective value: -1.2062444198387015
Epoch 20, Objective value: -1.2679602956778249
Epoch 30, Objective value: -1.2930044719338878
Epoch 40, Objective value: -1.3089965422814651
Epoch 50, Objective value: -1.3133206529961698
Epoch 60, Objective value: -1.3300882039041746
Epoch 70, Objective value: -1.3393339702537261
Epoch 80, Objective value: -1.3455036799675457
Epoch 90, Objective value: -1.3504263069219276
Epoch 100, Objective value: -1.3635356204844866
Epoch 110, Objective value: -1.3686326317958915
Epoch 120, Objective value: -1.3761389901748156
Epoch 130, Objective value: -1.3768341891163745
Epoch 140, Objective value: -1.3901542661835558
Epoch 150, Objective value: -1.3930511824861558
Epoch 160, Objective value: -1.397720693858103
Epoch 170, Objective value: -1.4037396593078681
Epoch 180, Objective value: -1.4108399931904705
Epoch 190, Objective value: -1.424721071625565
Epoch 200, Objective value: -1.4343570015714926
Epoch

In [130]:
c_sig

Automobiles & Components 1    0.106104
Banks 1                      -0.006668
Banks 2                       0.033130
Banks 3                       0.052938
Banks 4                       0.048104
                                ...   
Utilities 1319               -0.041928
Utilities 1320               -0.083635
Utilities 1321               -0.140086
Utilities 1322               -0.056594
Utilities 1323               -0.111543
Name: 2021-12-31 00:00:00, Length: 12752, dtype: float64

In [77]:
#TENSORFLOW EQUIVALENTS
ret = tf.reduce_sum(t * c_sig)
rev = lam_1 * tf.reduce_sum(t * c_rev)
c_w=tf.matmul(c_A,tf.expand_dims(t, axis=1))
tcosts = lam_2 *tf.reduce_sum(tf.abs(c_w-tf.expand_dims(c_w0,axis=1)))
vol=lam_3*tf.matmul(tf.matmul(c_w,c_cov,transpose_a=True),c_w)

In [ ]:
# Add penalty term for violating the constraints
def penalty(t):
    penalty = 0
    if np.abs(np.dot(c_A, t) @ np.ones(c_beta.shape)) > theta_1:
        penalty += 1e6 * np.abs(np.dot(c_A, t) @ np.ones(c_beta.shape) - theta_1)
    if np.abs(np.dot(c_A, t) @ c_beta) > theta_2:
        penalty += 1e6 * np.abs(np.dot(c_A, t) @ c_beta - theta_2)
    if np.abs(t) @ np.ones(t.shape) > theta_3:
        penalty += 1e6 * np.abs(np.abs(t) @ np.ones(t.shape) - theta_3)
    return penalty

# Update objective function with penalty
def obj_func_with_penalty(t):
    return obj_func(t) + penalty(t)

#NEED TO CREATE PENALTY FUNCTION IN TF

In [205]:
def obj_func(t):
    ret=(t@c_sig)
    rev=(lam_1*(t@c_rev))
    tcosts=(lam_2*sum(abs((c_A@t)-c_w0)))
    vol=(lam_3*((c_A@t).T@c_cov@(c_A@t)))
    return -(ret+rev-tcosts-vol)

theta_1=0.2 #Cash Neutrality
theta_2=0.2 #Beta Neutrality
theta_3=0.5 #Max Total Capital to All Trades
theta_4=0.01 #Max Cap per Trade

constraints = (
    {'type': 'ineq', 'fun': lambda t: theta_1 - abs((c_A@t)@np.ones(c_beta.shape))},  #partial cash neutrality
    {'type': 'ineq', 'fun': lambda t: theta_2 - abs((c_A@t)@c_beta)}, #partial beta neutrality
    {'type': 'ineq', 'fun': lambda t: theta_3 - abs(t)@np.ones(t.shape)} #max total cap to trades
)

bounds = [(-theta_4, theta_4)]*c_A.shape[1]

In [156]:
cash_neutrality=abs((c_A@t)@np.ones(c_beta.shape))
beta_neutrality=abs((c_A@t)@c_beta)
print(cash_neutrality,beta_neutrality)

0.00015527767124676693 0.001072796106645875


In [163]:
c_A.shape

(497, 41826)

In [158]:
abs(t)@np.ones(t.shape)

0.40000000000000047

In [ ]:
class optimizer:
    def __init__(self,data_obj,signal_dict,rev_dict,price_dict,filter_dict,coef_matrix):
        self.data=data_obj
        self.signal_dict=signal_dict
        self.rev_dict=rev_dict
        self.price_dict=price_dict
        self.filter_dict=filter_dict
        self.pnl=pnl(data_obj)
        self.coef_matrix=coef_matrix
    def optimize(self,port_val,c_w0,date_old,lam_1,lam_2,lam_3,theta_1,theta_2,theta_3):
        #Load all variables
        t_init=np.random.normal(size=sum(self.filter_dict[date_old]))
        t_init=t_init/(2.5*np.sum(np.abs(t_init)))
        c_sig=self.signal_dict[date_old][self.filter_dict[date_old]]
        c_rev=self.rev_dict[date_old][self.filter_dict[date_old]]
        c_A=weight_matrix(self.price_dict[date_old],self.coef_matrix).T
        c_cov=self.data.load_data(date_old,dtype='cov')
        c_beta=self.data.load_data(date_old,dtype='beta')
        
        def obj_func(t):
            #Maximize by minimize the negative
            ret=(t@c_sig)
            rev=(lam_1*(t@c_rev))
            tcosts=(lam_2*sum(abs((c_A@t)-c_w0)))
            vol=(lam_3*((c_A@t).T@c_cov@(c_A@t)))
            return -(ret+rev-tcosts-vol)
        
        constraints = (
            {'type': 'ineq', 'fun': lambda t: theta_1 - abs((c_A@t)@np.ones(c_beta.shape))},  #partial cash neutrality
            {'type': 'ineq', 'fun': lambda t: theta_2 - abs((c_A@t)@c_beta)}, #partial beta neutrality
            {'type': 'ineq', 'fun': lambda t: theta_3 - abs(t)@np.ones(t.shape)} #max total cap to trades
        )

        bounds = [(-theta_4, theta_4)]*c_A.shape[1]
        
        


In [118]:
print((c_A@t).T.shape)
print(c_cov.shape)

(497,)
(497, 497)


In [120]:
(c_A@t).T@c_cov@(c_A@t)

3.234473486516852e-07

In [48]:
import numpy as np
from scipy.optimize import minimize

# Define an objective function that takes a numpy array as input
def objective_function(x):
    # x is a numpy array; for example, let's minimize the sum of squares
    return np.sum(x**2)

# Initial guess for the numpy array
initial_guess = np.array([1.0, 2.0, 3.0])

# Call minimize with the objective function and initial guess
result = minimize(objective_function, initial_guess)

# Print the results
print("Optimized array:", result.x)
print("Objective function value:", result.fun)


Optimized array: [-1.27600518e-08 -8.68161543e-09 -3.55077334e-09]
Objective function value: 2.507973609166287e-16
